In [1]:
import skimage.io as io
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [2]:
# set path
base_path = os.path.join('..', '..', 'data', 'Gauss_1.0_0.3')

# initialize lists
tracker_list = []
emcal_list = []
hcal_list = []
truth_list = []

In [3]:
# read images and append to lists
for i in range(1000):  
    tracker_image_path = os.path.join(base_path, f'tracker_{i}.tiff')
    tracker = io.imread(tracker_image_path)
    tracker_list.append(tracker)

    emcal_image_path = os.path.join(base_path, f'emcal_{i}.tiff')
    emcal = io.imread(emcal_image_path)
    emcal_list.append(emcal)

    hcal_image_path = os.path.join(base_path, f'hcal_{i}.tiff')
    hcal = io.imread(hcal_image_path)
    hcal_list.append(hcal)
    
    truth_image_path = os.path.join(base_path, f'truth_{i}.tiff')
    truth = io.imread(truth_image_path)
    truth_list.append(truth)

In [4]:
# convert lists to numpy arrays
X_tracker = np.array(tracker_list).flatten()
X_emcal = np.array(emcal_list).flatten()
X_hcal = np.array(hcal_list).flatten()
Y_truth = np.array(truth_list).flatten()

In [5]:
# concatenate feature arrays
X=np.array(list(zip(X_tracker,X_emcal,X_hcal )))
Y = Y_truth

In [6]:
# split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, Y_train)

# predict on the test set
Y_pred = model.predict(X_test)

print(model.score(X_test,Y_test ))


0.9879353647368017


In [7]:
sum = 0
for image_index in range(100):
    image_size = 56 * 56
    start_index = image_index * image_size
    end_index = start_index + image_size

    true_data = Y_test[start_index:end_index]
    predicted_data = Y_pred[start_index:end_index]

    true_picture = np.array(true_data).reshape((56, 56))
    predicted_picture = np.array(predicted_data).reshape((56, 56))
    # Replace zero values in true_data and predicted_data with a small number to avoid division by zero
    true_data = np.where(true_data == 0, 1e-20, true_data)

    # Calculate the error metrics
    error_2d = predicted_data - true_data
    error_1d = predicted_data - true_data
    rms_2d = np.sqrt(np.mean(error_2d**2))
    sum +=rms_2d
print(sum/100)

0.11087609074581622
